In [227]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

# Cleaning the data (modelling steps)

In [228]:
#import dataset from EDA
df =pd.read_pickle(r'C:\DataScienceProjects\201907_Churn_ Prediction\3. Uploaded Data\churn_data_for_modelling.pkl')

In [229]:
# assigning user ids to a variable and removing from the dataset
user_id = df['user']
df = df.drop(columns=['user'])

In [230]:
# one hot encoding of categorical variables
df = pd.get_dummies(df)

In [231]:
# removing correlated columns created with one hot encoding, i.e. 'na' columns which included the null values
df = df.drop(columns = ['housing_na','payment_type_na','zodiac_sign_na'])

# Train_test split

In [232]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns =['churn']), # independent variables
                                                df['churn'],      # dependent variable
                                                test_size=0.2,
                                                random_state=0)

In [233]:
# Checking the balancing of the dataset
y_train.value_counts()

0    12656
1     8940
Name: churn, dtype: int64

# Rebalancing dataset

In [234]:
#Dividing dataset by class
y_train_neg = y_train[y_train.values==0].index
y_train_pos = y_train[y_train.values==1].index

In [235]:
# Under-sampling of classes in relation to the higher represented class
if len(y_train_neg)>len(y_train_pos):
    higher = y_train_neg
    lower = y_train_pos
else:
    higher = y_train_pos
    lower = y_train_

np.random.seed(0)
higher = np.random.choice(higher, size = len(lower)) # pick randomly a subset of observations (rows) matching the size of the smallest class
lower = np.asarray(lower) #converting to an array
new_indexes = np.concatenate((lower,higher)) # concatenating arrays of indexes

# undersampling training data
X_train = X_train.loc[new_indexes,]  #all columns
y_train = y_train.loc[new_indexes]   #only one col



In [236]:
y_train.value_counts()

1    8940
0    8940
Name: churn, dtype: int64

# Scaling

In [237]:
from sklearn.preprocessing import StandardScaler

In [238]:
sc_X = StandardScaler()
# build a "scaled" training set
X_train2 = pd.DataFrame (sc_X.fit_transform(X_train))
X_test2 = pd.DataFrame (sc_X.transform(X_test)) 

# copying the columns name to the scaled dataset
X_train2.columns = X_train.columns.values 
X_test2.columns = X_test.columns.values

# copying the index to the scaled dataset.
X_train2.index = X_train.index.values 
X_test2.index = X_test.index.values

X_train = X_train2
X_test = X_test2

# Model building :

## Logistic Regression (all features)

In [239]:
from sklearn.linear_model import LogisticRegression
classifier_full = LogisticRegression(random_state=0,solver='lbfgs')
classifier_full.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [240]:
y_pred = classifier_full.predict(X_test)

In [241]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score, classification_report 
#Evaluating performance metrics
def model_evaluation(y_test,y_pred):
    cm = confusion_matrix(y_test,y_pred)
    df_cm = pd.DataFrame(cm, index = ('False','True'), columns = ('False','True'))  
    fig = ff.create_annotated_heatmap(x=df_cm.columns.values.tolist(),
                                      y=df_cm.index.values.tolist(),
                                      z=df_cm.values.tolist(),
                                      colorscale='Spectral',
                                     showscale=True)
    fig.update_layout(title={'text':'Confusion Matrix',
                                'font':{'family':'Merriweather',
                                        'size':24},
                               },
                       xaxis = {"title":{"text":"Predicted Value",
                                         "font":{'family':'open sans',
                                                 'size':18}}},                   
                       yaxis = {"title":{"text":"True Value",
                                         "font":{'family':'open sans',
                                                 'size':18}}},
                      template="plotly_dark"
                      )
    fig.show()
    print(f'Accuracy score: {round(accuracy_score(y_test,y_pred)*100,2)}%')
    print(f'Precision score: {round(precision_score(y_test,y_pred)*100,2)}%')
    print(f'Recall score: {round(recall_score(y_test,y_pred)*100,2)}%')
    print(f'F1 score: {round(f1_score(y_test,y_pred)*100,2)}%')
    print(classification_report(y_test,y_pred))

In [242]:
model_evaluation(y_test,y_pred)

Accuracy score: 61.3%
Precision score: 52.28%
Recall score: 73.81%
F1 score: 61.21%
              precision    recall  f1-score   support

           0       0.74      0.52      0.61      3166
           1       0.52      0.74      0.61      2234

    accuracy                           0.61      5400
   macro avg       0.63      0.63      0.61      5400
weighted avg       0.65      0.61      0.61      5400



In [243]:
# Applying k-fold Cross Validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier,
                            X = X_train,
                            y = y_train,
                            cv =10)
accuracies.mean()

0.6378635346756153

In [244]:
# Analyzing coefficient of the features in the decision function
features = pd.DataFrame(X_train.columns,columns=['features']) # build a dataframe with column name in rows
coefficient = pd.DataFrame(classifier_full.coef_.T,columns = ["coef"]) #build a dataframe with coefficients
df_coef = pd.concat([features,coefficient],axis=1)

In [223]:
df_coef.sort_values('coef',ascending=False)

,features,coef
12,ios_user,0.126610
11,web_user,0.114792
18,rejected_loan,0.114086
17,received_loan,0.110471
14,registered_phones,0.093949
16,cancelled_loan,0.091392
13,android_user,0.070288
6,cc_recommended,0.068813
19,left_for_two_month_plus,0.056513
5,cc_taken,0.051114


In [245]:
#Feature selection
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression


In [246]:
#Model To be tested
clf = LogisticRegression(solver='lbfgs')
rfe = RFE(clf, 20) #using Recursive Feature elimination
rfe = rfe.fit(X_train,y_train)

X_train.columns[rfe.support_] #display important features


Index(['age', 'withdrawal', 'purchases_partners', 'purchases', 'cc_taken',
       'cc_recommended', 'cc_application_begin', 'app_downloaded', 'web_user',
       'ios_user', 'android_user', 'registered_phones', 'waiting_4_loan',
       'cancelled_loan', 'received_loan', 'rejected_loan',
       'left_for_two_month_plus', 'left_for_one_month', 'reward_rate',
       'payment_type_Weekly'],
      dtype='object')

## Logistic Regression (selected features)

In [180]:
# Training the model with selected features
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state=0,solver='lbfgs')
classifier.fit(X_train[X_train.columns[rfe.support_]],y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [181]:
y_pred = classifier.predict(X_test[X_test.columns[rfe.support_]])

In [182]:
model_evaluation(y_test,y_pred)

Accuracy score: 60.98%
Precision score: 52.0%
Recall score: 73.9%
F1 score: 61.05%
              precision    recall  f1-score   support

           0       0.74      0.52      0.61      3166
           1       0.52      0.74      0.61      2234

    accuracy                           0.61      5400
   macro avg       0.63      0.63      0.61      5400
weighted avg       0.65      0.61      0.61      5400



Although the perofrmance of the model did not increase a lot we managed to slice by half the predictors and get almost the same results than the previous model containing all the features.
This improve our ability to understand what are the key features leading users to churn.

In [183]:
# Analyzing coefficients:
features = pd.DataFrame(X_train.columns[rfe.support_],columns=['features']) # build a dataframe with column name in rows
coefficient = pd.DataFrame(classifier.coef_.T,columns = ["coef"]) #build a dataframe with coefficients
df_coef_short = pd.concat([features,coefficient],axis=1)

In [184]:
df_coef_short

,features,coef
0,age,-0.165304
1,withdrawal,0.044562
2,purchases_partners,-0.675173
3,purchases,-0.150695
4,cc_taken,0.050427
5,cc_recommended,0.072417
6,cc_application_begin,0.046789
7,app_downloaded,-0.045413
8,web_user,0.116577
9,ios_user,0.133312


In [52]:
# Formatting final_results
final_results = pd.concat([y_test,user_id], axis=1).dropna()
final_results['predicted_churn'] = y_pred
final_results = final_results[['user','churn','predicted_churn']].reset_index(drop=True)

In [53]:
final_results

,user,churn,predicted_churn
0,61353,1.0,1
1,67679,0.0,0
2,21269,0.0,0
3,69531,0.0,1
4,25997,0.0,0
...,...,...,...
5395,22377,0.0,0
5396,24291,1.0,1
5397,23740,0.0,1
5398,47663,1.0,0


Conclusion:

Model gives us an indication of which users are more likely to churn. It is important to note that we have not contrained our model to a timeframe by setting the expected churn date as we want to have an idea of what features are the signs of a disengagement.
Indeed, we won't necessarily built new products or features for users who are almost sure to leave but for the ones who are starting to lose interest.
If, after the release of new products / features, we start seeing our model predicting that less of our users are going to churn, then we can assume our customers are feeling more engaged. As a follow-up, we could release a poll to get more insights on how they perceived our new features.

To strengthen the accuracy of our model we can think about setting up a time dimension to churn.


# Bonus


## Random Forest

In [264]:
from sklearn.ensemble import RandomForestClassifier
N_ESTIMATORS = 500
MAX_DEPTH = 6
RFClf=RandomForestClassifier(n_estimators=N_ESTIMATORS,max_depth=MAX_DEPTH)
RFClf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=6, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [265]:
y_pred = RFClf.predict(X_test)

In [266]:
model_evaluation(y_test,y_pred)

Accuracy score: 64.24%
Precision score: 55.54%
Recall score: 68.04%
F1 score: 61.15%
              precision    recall  f1-score   support

           0       0.73      0.62      0.67      3166
           1       0.56      0.68      0.61      2234

    accuracy                           0.64      5400
   macro avg       0.64      0.65      0.64      5400
weighted avg       0.66      0.64      0.65      5400



## Additionnal Metrics: ROC Curve

In [267]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
#Prediction probabilities for the test data
probs = RFClf.predict_proba(X_test)
probs = probs[:,1] #keep only the prediction for the positive class
# Compute AUC Score
auc = roc_auc_score(y_test,probs)
# ROC curve data
fpr, tpr, thresholds = roc_curve(y_test, probs)

In [268]:
trace_ROC = go.Scatter(x=fpr,y=tpr,
                   mode='lines',
                       name='ROC Curve (Area={})'.format(round(auc,3)))
trace_neutral = go.Scatter(x=[0, 1],y=[0, 1],
                   mode='lines',
                          name='No skill curve')

data=[trace_ROC,trace_neutral]
layout=go.Layout(title={'text':'Receiver operating Characteristic (ROC) Curve',
                                'font':{'family':'Merriweather',
                                        'size':24},
                               },
                       xaxis = {"title":{"text":"False Positive Rate",
                                         "font":{'family':'open sans',
                                                 'size':18}}},                   
                       yaxis = {"title":{"text":"True Positive Rate",
                                         "font":{'family':'open sans',
                                                 'size':18}}},
                template="plotly_dark")
                      

fig = go.Figure(data=data,layout=layout)

fig.show()

In [269]:
coefficient_rf=pd.DataFrame(RFClf.feature_importances_, columns=['rf_coef'])
features_rf=pd.DataFrame(X_test.columns,columns=['features'])
rf_coeff=pd.concat([features_rf,coefficient_rf],axis=1).sort_values(['rf_coef'],ascending=True)

In [270]:
#plotting features importance
fig = px.bar(rf_coeff, x="rf_coef", y="features", orientation='h')
fig.update_layout(autosize=False,
                         width=1000,
                         height=800,
                         template="plotly_dark")
fig.show()

In [271]:
#Let's compare the features from RandomForest with Logistic Regression
df_coef['coef_abs']=abs(df_coef['coef'])

In [272]:
df_coef_abs=df_coef.drop(columns=['coef'])

In [273]:
df_coeff_abs=df_coef_abs.merge(rf_coeff,how='inner',left_on='features',right_on='features').sort_values(['coef_abs'])

In [274]:
trace0=go.Bar(x=df_coeff_abs['features'],
            y=df_coeff_abs['coef_abs'],
            name='LogReg Coefficients')
trace1=go.Bar(x=df_coeff_abs['features'],
            y=df_coeff_abs['rf_coef'],
            name='RandomForest Coefficients')
data=[trace0,trace1]
fig=go.Figure(data=data)
fig.show()

There is a discrepancy between the results. Let's see what are the results of a permutation importance

In [275]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(RFClf, random_state=1).fit(X_test, y_test)
eli5.show_weights(perm, feature_names = X_test.columns.tolist())

Weight,Feature
0.0860 ± 0.0098,purchases_partners
0.0249 ± 0.0042,reward_rate
0.0150 ± 0.0032,cc_recommended
0.0029 ± 0.0018,cc_taken
0.0021 ± 0.0011,age
0.0018 ± 0.0008,cc_application_begin
0.0017 ± 0.0016,left_for_two_month_plus
0.0017 ± 0.0019,purchases
0.0015 ± 0.0019,withdrawal
0.0014 ± 0.0013,web_user


In [276]:
perm = PermutationImportance(classifier_full, random_state=0).fit(X_test, y_test)
eli5.show_weights(perm, feature_names = X_test.columns.tolist())

Weight,Feature
0.0927 ± 0.0063,purchases_partners
0.0076 ± 0.0070,age
0.0037 ± 0.0028,registered_phones
0.0030 ± 0.0011,left_for_two_month_plus
0.0025 ± 0.0012,zodiac_sign_Scorpio
0.0024 ± 0.0033,deposits
0.0019 ± 0.0003,rejected_loan
0.0019 ± 0.0012,cc_taken
0.0018 ± 0.0010,payment_type_Semi-Monthly
0.0017 ± 0.0016,received_loan


Different methods are providing different results in terms of which features impact the most our models. Some independent variables are consistently ranked among the top predictors.
These variations and inconsitencies are to some extent related to the multicolinerity among independent variables.
To further fine tune our recommendations we could further investigate on removing the multicolinearity we observed.